In [ ]:
!pip install datasets -U
!pip install transformers -U
!pip install torchaudio -U
!pip install librosa -U
!pip install jiwer -U
!apt install git-lfs -U
!pip install hazm -U
!pip install pydub -U
!pip install pythainlp -U
!pip install huggingface-hub -U
!pip install num2fawords -U
!pip install parsivar -U

     |████████████████████████████████| 346 kB 8.3 MB/s 
     |████████████████████████████████| 1.1 MB 53.2 MB/s 
     |████████████████████████████████| 84 kB 3.9 MB/s 
     |████████████████████████████████| 140 kB 55.4 MB/s 
     |████████████████████████████████| 212 kB 73.2 MB/s 
     |████████████████████████████████| 127 kB 65.9 MB/s 
     |████████████████████████████████| 271 kB 70.1 MB/s 
     |████████████████████████████████| 144 kB 76.9 MB/s 
     |████████████████████████████████| 94 kB 3.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.2 MB 7.2 MB/s 


In [ ]:
import os
import re
import pandas as pd
import torch
import os
import random
import numpy as np
import string
import json
from IPython.display import display
import librosa
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers.trainer_utils import get_last_checkpoint
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2FeatureExtractor, Wav2Vec2Config, TrainingArguments, Trainer
from parsivar import Normalizer
import num2fawords
import re
import string
from typing import Any, Dict, List, Optional, Union
from dataclasses import dataclass

In [ ]:
from google.colab import drive


drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!wget https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/fa.tar.gz

!tar -xzf fa.tar.gz
!rm -rf fa.tar.gz

--2022-05-23 06:27:51--  https://voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com/cv-corpus-6.1-2020-12-11/fa.tar.gz
Resolving voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)... 52.218.237.27
Connecting to voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com (voice-prod-bundler-ee1969a6ce8178826482b88e843c335139bd3fb4.s3.amazonaws.com)|52.218.237.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8884585819 (8.3G) [application/octet-stream]
Saving to: ‘fa.tar.gz’

fa.tar.gz           100%[===================>]   8.27G  81.4MB/s    in 2m 3s   

2022-05-23 06:29:54 (69.1 MB/s) - ‘fa.tar.gz’ saved [8884585819/8884585819]



In [ ]:
_normalizer = Normalizer(half_space_char="\u200c", statistical_space_correction=True)

chars_to_ignore = [
    ",", "?", ".", "!", "-", ";", ":", '""', "%", "'", '"', "�",
    "#", "!", "؟", "?", "«", "»", "،", "(", ")", "؛", "'ٔ", "٬", 'ٔ', ",", "?",
    ".", "!", "-", ";", ":", '"', "“", "%", "‘", "”", "�", "–", "…", "_", "”", '“', '„',
    'ā', 'š', 'ّ', 'ْ',
]

chars_to_ignore = chars_to_ignore + list(string.ascii_lowercase + string.digits)
chars_to_ignore = f"""[{"".join(chars_to_ignore)}]"""
zwnj = "\u200c"
silent_chars = ["ا", "د", "ذ", "ر", "ز", "و", "آ"] + [zwnj] + [" "]


def multiple_replace(text, chars_to_mapping):
    pattern = "|".join(map(re.escape, chars_to_mapping.keys()))
    return re.sub(pattern, lambda m: chars_to_mapping[m.group()], str(text))


def remove_special_characters(text, chars_to_ignore_regex):
    text = re.sub(chars_to_ignore_regex, '', text).lower() + " "
    return text


def convert_word_nums_to_text(word):
    try:
        word = int(word)
        word = num2fawords.words(word)
    except:
        word = word

    return word


def normalizer_at_word_level(text):
    words = text.split()
    _text = []

    for word in words:
        word = convert_word_nums_to_text(word)
        word = fixator_dictionary.get(word, word)

        _text.append(word)

    return " ".join(_text) + " "


def finder(ss, s, starter=False):
    found = []
    for m in re.finditer(ss, s):
        if starter:
            found.append(m.start())
        else:
            found.append((m.start(), m.end()))

    return found


def substring_replace(ss, s, start, end, stripped=True):
    s_start = s[:start]
    s_end = s[end:]

    counter = 0
    if stripped:
        counter = 1 if s_start.endswith(" ") else counter
        s_start = s_start.rstrip()

    return s_start + ss + s_end, counter


def normalizer(
        batch,
        is_normalize=True,
        return_dict=True,
        filter_trivials=False,
        remove_extra_space=False
):
    text = batch["sentence"].lower().strip()

    # Parsivar normalizer
    if is_normalize:
        text = _normalizer.normalize(text)

    # Dictionary mapping
    text = multiple_replace(text, dictionary_mapping)
    text = re.sub(" +", " ", text)

    # Remove specials
    text = remove_special_characters(text, chars_to_ignore)
    text = re.sub(" +", " ", text)

    # Replace connected آ
    special, pointer = "آ", int("0")
    for f in sorted(finder(special, text, True)):
        index = f + pointer - 1
        if len(text) >= index:
            if text[index] not in silent_chars:
                new_text, extra_pointer = substring_replace(
                    f"{text[index]}{zwnj}", text, index, index + 1, stripped=True)
                text = new_text
                pointer += 1 + 1 - 1 - extra_pointer

    # Replace connected ها
    pointer = int("0")
    special_list = [
        # "ام", "ای", "است", "ایم", "اید", "اند",
        "هایمان", "هایم", "هایت", "هایش",
        "هایتان", "هایشان", "هام", "هات",
        "هاتان", "هامون", "هامان", "هاش",
        "هاتون", "هاشان", "هاشون",
        "هایی", "های", "هاس", "ها"
    ]
    for special in special_list:
        pointer = 0
        text = text
        for f in sorted(finder(special, text, False)):
            start, end = f[0] + pointer - 1, f[1] + pointer - 1
            if len(text) >= (end + 1):
                if len(text) == (end + 1):
                    new_text, extra_pointer = substring_replace(
                        f"{zwnj}{special}",
                        text,
                        start + 1,
                        end + 1,
                        stripped=True)
                    text = new_text
                    pointer += 1 + 1 - 1 - extra_pointer
                else:
                    if text[end + 1] == " ":
                        new_text, extra_pointer = substring_replace(
                            f"{zwnj}{special}",
                            text,
                            start + 1,
                            end + 1,
                            stripped=True)
                        text = new_text
                        pointer += 1 + 1 - 1 - extra_pointer

    special, pointer = "افزار", int("0")
    for f in sorted(finder(special, text, False)):
        start, end = f[0] + pointer - 1, f[1] + pointer - 1

        if len(text) >= (end + 1):
            new_text, extra_pointer = substring_replace(f"{zwnj}{special}", text, start + 1, end + 1, stripped=True)
            text = new_text
            pointer += 1 + 1 - 1 - extra_pointer

    # Replace connected ها
    pointer = int("0")
    special_list = [
        "ترین", "تر"
    ]
    for special in special_list:
        pointer = 0
        text = text
        for f in sorted(finder(special, text, False)):
            start, end = f[0] + pointer - 1, f[1] + pointer - 1
            if len(text) >= (end + 1):
                if len(text) == (end + 1):
                    new_text, extra_pointer = substring_replace(
                        f"{zwnj}{special}",
                        text,
                        start + 1,
                        end + 1,
                        stripped=True)
                    text = new_text
                    pointer += 1 + 1 - 1 - extra_pointer
                else:
                    if text[end + 1] == " ":
                        new_text, extra_pointer = substring_replace(
                            f"{zwnj}{special}",
                            text,
                            start + 1,
                            end + 1,
                            stripped=True)
                        text = new_text
                        pointer += 1 + 1 - 1 - extra_pointer

    # Normalizer at word level
    text = normalizer_at_word_level(text)
    text = re.sub(" +", " ", text)

    if remove_extra_space:
        text = text.strip()
    else:
        text = text.strip() + " "

    if filter_trivials:
        if not len(text) > 2:
            text = None

    if not return_dict:
        return text

    batch["sentence"] = text
    return batch

In [ ]:
dictionary_mapping = {
    'ك': 'ک', 'دِ': 'د', 'بِ': 'ب', 'زِ': 'ز', 'ذِ': 'ذ', 'شِ': 'ش', 'سِ': 'س', 'ى': 'ی',
    'ي': 'ی', 'أ': 'ا', 'ؤ': 'و', "ے": "ی", "ۀ": "ه", "ﭘ": "پ", "ﮐ": "ک", "ﯽ": "ی",
    "ﺎ": "ا", "ﺑ": "ب", "ﺘ": "ت", "ﺧ": "خ", "ﺩ": "د", "ﺱ": "س", "ﻀ": "ض", "ﻌ": "ع",
    "ﻟ": "ل", "ﻡ": "م", "ﻢ": "م", "ﻪ": "ه", "ﻮ": "و", 'ﺍ': "ا", 'ة': "ه",
    'ﯾ': "ی", 'ﯿ': "ی", 'ﺒ': "ب", 'ﺖ': "ت", 'ﺪ': "د", 'ﺮ': "ر", 'ﺴ': "س", 'ﺷ': "ش",
    'ﺸ': "ش", 'ﻋ': "ع", 'ﻤ': "م", 'ﻥ': "ن", 'ﻧ': "ن", 'ﻭ': "و", 'ﺭ': "ر", "ﮔ": "گ",

    "a": "‌ای‌", "b": "‌بی‌", "c": "‌سی‌", "d": "‌دی‌", "e": "‌ایی‌", "f": "‌اف‌",
    "g": "‌جی‌", "h": "‌اچ‌", "i": "‌آی‌", "j": "‌جی‌", "k": "‌کی‌", "l": "‌ال‌",
    "m": "‌ام‌", "n": "‌ان‌", "o": "‌او‌", "p": "‌پی‌", "q": "‌کیو‌", "r": "‌آر‌",
    "s": "‌اس‌", "t": "‌تی‌", "u": "‌یو‌", "v": "‌وی‌", "w": "‌دبلیو‌", "x": "‌اکس‌",
    "y": "‌وای‌", "z": "‌زد ",
    "\u200c": " ", "\u200d": " ", "\u200e": " ", "\u200f": " ", "\ufeff": " ",

    "نو آوری‌مان": "نو‌آوری‌مان",
    "نو آوری مان": "نو‌آوری‌مان",
    "نو آوریمان": "نو‌آوری‌مان",
    " ا م ": "‌ام ",
    " م ": "‌ام ",
    "کنندهای": "کننده‌ای",
    "ارائهای": "ارائه‌ای",
    "ایدهای": "ایده‌ای",
    "ماسهای": "ماسه‌ای",
    "خامنهای": "خامنه‌ای",
    "قلهای": "قله‌ای",
    "سیارهای": "سیاره‌ای",
    "کیسهای": "کیسه‌ای",
    "شانهای": "شانه‌ای",
    "غریبهای": "غریبه‌ای",
    "برنامهای": "برنامه‌ای",
    "سختگیرانهای": "سختگیرانه‌ای",
    "بهانهای": "بهانه‌ای",
    "زیرروالهای": "زیر روالهای",
    "درهای": "دره‌ای",
    "آمادهای": "آماده‌ای",
    "سادهای": "ساده‌ای",
    "سرمایهگذارهای": "سرمایه گذارهای",
    "فوقالعادهای": "فوق‌العاده‌ای",
    "حادثهای": "حادثه‌ای",
    "نویسندههای": "نویسنده‌های",
    "علاقهای": "علاقه‌ای",
    "برجستهای": "برجسته‌ای",
    "جلگهای": "جلگه‌ای",
    "زندهای": "زنده‌ای",
    "فنآوریهای": "فناوری‌های",
    "سایهروشنهای": "سایه روشن‌های",
    "بیسابقهای": "بی سابقه‌ای",
    "فرضیهای": "فرضیه‌ای",
    "راهاندازهای": "راه اندازهای",
    "بیشهای": "بیشه‌ای",
    "مقالهای": "مقاله‌ای",
    "دیگهای": "دیگه‌ای",
    "ماههاست": "ماه هاست",
    "نرمافزارهای": "نرم‌افزارهای",
    "کتابسوزانهای": "کتاب سوزان‌های",
    "سیستمعاملهای": "سیستم عامل‌های",
    "اسلحهای": "اسلحه‌ای",
    "وقفهای": "وقفه‌ای",
    "زمینهای": "زمینه‌ای",
    "حرامزادههای": "حرامزاده‌های",
    "هزینهای": "هزینه‌ای",
    "انداختهای": "انداخته‌ای",
    "جسورانهای": "جسورانه‌ای",
    "فاجعهای": "فاجعه‌ای",
    "جامعهای": "جامعه‌ای",
    "پدیدهای": "پدیده‌ای",
    "اغواگرانهای": "اغواگرانه‌ای",
    "تکانهای": "تکانه‌ای",
    "لولهای": "لوله‌ای",
    "نشانهای": "نشانه‌ای",
    "وسیلهای": "وسیله‌ای",
    "آیندهای": "آینده‌ای",
    "بردهای": "برده‌ای",
    "سابقهای": "سابقه‌ای",
    "ناحیهای": "ناحیه‌ای",
    "تکاندهندهای": "تکان دهنده‌ای",
    "بودجهای": "بودجه‌ای",
    "روزانهای": "روزانه‌ای",
    "چارهای": "چاره‌ای",
    "انگیزهای": "انگیزه‌ای",
    "دادهای": "داده‌ای",
    "عدهای": "عده‌ای",
    "هفتهای": "هفته‌ای",
    "منطقهای": "منطقه‌ای",
    "استارتآپهای": "استارتاپ‌های",
    "سازهای": "سازه‌ای",
    "مجموعهای": "مجموعه‌ای",
    "فلسفهای": "فلسفه‌ای",
    "تذکردهندهای": "تذکر دهنده‌ای",
    "مصاحبهای": "مصابحه‌ای",
    "نمونهای": "نمونه‌ای",
    "قلمموهای": "قلم مو‌های",
    "شبزندهداری": "شب زنده‌داری",
    "خوردهباشد": "خورده باشد",
    "داشتهباشید": "داشته باشید",
    "فزایندهای": "فزاینده‌ای",
    "عمدهای": "عمده‌ای",
    "بدیهایی": "بدی‌های",
    "نوشت‌هایم": "نوشته‌ایم",
    "بنتالهدی": "بنت الهدی",
    "نوشتهام": "نوشته‌ام",
    "سرمایهگذاران": "سرمایه گذاران",
    "خانهی": "خانه‌ی",
    "گستاخانهی": "گستاخانه‌ی",
    "گرفتهباشیم": "گرفته باشیم",
    "خونهی": "خونه‌ی",
    "داشتهام": "داشته‌ام",
    "رشتهام": "رشته‌ام",
    "سرمایهگذارانشان": "سرمایه گذارانشان",
    "ریشهکنی": "ریشه‌کنی",
    "مودبانهتری": "مودبانه‌تری",
    "برگردانشدهاند": "برگردان شده‌اند",
    "قرمهسبزی": "قرمه‌سبزی",
    "راهجویی": "راه جویی",
    "اماهیچوقت": "اما هیچوقت",
    "آبوهوای": "آب و هوای",
    "بقیهاش": "بقیه‌اش",
    "طبقهبندی": "طبقه‌بندی",
    "مردههان": "مرده هان",
    "آمادهاند": "آماده‌اند",
    "نشدهاید": "نشده‌اید",
    "آگاهیرسانی": "آگاهی رسانی",
    "نداشتهاند": "نداشته‌اند",
    "شکنانهترین": "شکنانه‌ترین",
    "اقدامهایی": "اقدام‌هایی",
    "راهآهن": "راه آهن",
    "شدهاند": "شده‌اند",
    "تازهترین": "تازه‌ترین",
    "روبهروی": "رو به روی",
    "منحصربهفرد": "منحصر به فرد",
    "سیزدهبدر": "سیزده بدر",
    "برندهی": "برنده‌ی",
    "خانهاشتراکی": "خانه اشتراکی",
    "دادههایی": "داده‌هایی",
    "استفادهتر": "استفاده‌تر",
    "گذرنامهتان": "گذرنامه‌تان",
    "کهنترین": "کهنه‌ترین",
    "فرهنگسرا": "فرهنگ‌سرا",
    "آمادهاید": "آماده‌اید",
    "ویژهی": "ویژه‌ی",
    "غریزهات": "غریزه‌ات",
    "مادرشوهری": "مادر شوهری",
    "نبودهام": "نبوده‌ام",
    "بودهاند": "بوده‌اند",
    "وتنها": "و تنها",
    "بداههکاری": "بداهه‌کاری",
    "سرمایهگذار": "سرمایه گذار",
    "برنامهنویس": "برنامه نویس",
    "مهنازخانم": "مهناز خانم",
    "مواجهاند": "مواجه‌اند",
    "توسعهاش": "توسعه‌اش",
    "سینهام": "سینه‌ام",
    "سین‌هام": "سینه‌ام",
    "نمیخواهند": "نمیخواهند",
    "فنآوری‌ها": "فناوری‌ها",
    "دنبالهرو": "دنباله‌رو",
    "لبهی": "لبه‌ی",
    "اللهیار": "الله یار",
    "ارزندهتر": "ارزنده‌تر",
    "برههای": "بره‌ای",
    "پیادهسازی": "پیاده‌سازی",
    "دهسالگی": "ده سالگی",
    "رسانهای": "رسانه‌ای",
    "ریشسفیدها": "ریش سفید‌ها",
    "چهجوری": "چه جوری",
    "ویژگیهایی": "ویژگی‌هایی",
    "می‌فهمی‌م": "میفهمیم",
    "وبهم": "و بهم",
    "قطرهای": "قطره‌ای",
    "ازتنهایی": "از تنهایی",
    "لطیفهای": "لطیفه‌ای",
    "باشهاومدم": "باشه اومدم",
    "منحصربهفردترین": "منحصر به فرد‌ترین",
    "کردهاند": "کرده‌اند",
    "اندازهای": "اندازه‌ای",
    "بهرهبرداری": "بهره برداری",
    "اماشوهرجان": "اما شوهر جان",
    "خانوادهاش": "خانواده‌اش",
    "نشدهاند": "نشده‌اند",
    "نکردهایم": "نکرده‌ایم",
    "تخممرغ‌هایش": "تخم مرغ‌هایش",
    "وظیفهش": "وظیفه‌اش",
    "مشگینشهر": "مشگی شهر",
    "توسعهدهندگانش": "توسعه دهندگانش",
    "امینابراهیم": "امین ابراهیم",
    "دربارهاش": "درباره‌اش",
    "میانافزارها": "میان‌افزارها",
    "دیدهاند": "دیده‌اند",
    "خانوادهام": "خانواده‌ام",
    "مایهی": "مایه‌ی",
    "نوشتهشدن": "نوشته شدن",
    "راهحل‌هایشان": "راه حل‌هایشان",
    "میهماننواز": "میهمان نواز",
    "زیبندهی": "زیرنده‌ی",
    "راههایی": "راه‌هایی",
    "جربزهی": "جربزه‌ی",
    "بهجا": " به جا",
    "بطورهمزمان": "به طور همزمان",
    "فهمیدهبود": "فهمیده بود",
    "دوربرگردان‌ها": "دور برگردان‌ها",
    "شالودهی": "شالوده‌ی",
    "راهکاریی": "راه‌کاری",
    "مخالفتهایی": "مخالفت‌هایی",
    "چیزهاازشون": "چیزها ازشون",
    "سکونتگاه‌های": "سکونت گاه‌های",
    "سالهابود": "سال‌ها بود",
    "نمونهی": "نمونه‌ی",
    "سرمایهگذاری": "سرمایه گذاری",
    "شبکهای": "شبکه‌ای",
    "خواهرشوهر": "خواهر شوهر",
    "سرگیجهآور": "سرگیجه آور",
    "آستانهی": "آستانه‌ی",
    "دادهاست": "داده است",
    "مجسمهسازی": "مجسمه سازی",
    "ماهرانهترین": "ماهرانه‌ترین",
    "پنجشنبههایی": "پنجشنبه شب‌هایی",
    "نرفنهام": "نرفته‌ام",
    "قورمهسبزی": "قورمه سبزی",
    "گذارهای": "گذاره‌ای",
    "بندهخدا": "بنده خدا",
    "روزنامهنگاران": "روزنامه نگاران",
    "نقشهی": "نقشه‌ی",
    "حملهی": "حمله‌ی",
    "تکنیکهاست": "تکنیک هاست",
    "نرمافزارهایمان": "نرم‌افرارهایمان",
    "مادرشوهرم": "مادر شوهرم",
    "ماهگیمون": "ماه گیمون",
    "مادرشوهرمحترم": "مادر شوهر محترم",
    "شوهرداری": "شوهر داری",
    "سرمایهگذارها": "سرمایه گذارها",
    "بهرهمند": "بهره‌مند",
    "درمانهایی": "درمان‌هایی",
    "عامدانهتر": "عامدانه‌تر",
    "تازهوارد": "تازه وارد",
    "مونتهویدئو": "مونته ویدئو",
    "ذائق‌هاش": "ذائقه‌اش",
    "گوشهگیرتر": "گوشه‌گیرتر",
    "دنبالهدار": "دنباله‌دار",
    "بیخانمان‌ها": "بی‌خانمان‌ها",
    "سرمایهدارها": "سرمایه‌دارها",
    "مادرشوهریم": "مادر شوهریم",
    "صبحان‌هاش": "صبحانه‌اش",
    "جنازهست": "جنازه است",
    "شمارهات": "شماره‌ای",
    "بهقدری": "به قدری",
    "کیسهی": "کیسه‌ی",
    "کوششهایی": "کوشش‌هایی",
    "مادرشوهر": "مادر شوهر",
    "رابطهی": "رابطه‌ی",
    "نوشتهاند": "نوشته‌اند",
    "کنجکاوانهی": "کنجکاوانه‌ی",
    "غیرمتعهد": "غیر متعهد",
    "کردهای": "کرده‌ای",
    "وهمکارانم": "و همکارانم",
    "گردهمآیی": "گردهمایی",
    "اللهوردی": "الله وردی",
    "صرفهجویی": "صرفه جویی",
    "ماندهاند": "مانده‌اند",
    "برنامهنویسی": "برنامه‌نویسی",
    "امینمهدی": "امین مهدی",
    "سهامدارنی": "سهام دارانی",
    "مسابقهی": "مسابقه‌ی",
    "ستارهشناسم": "ستار شناسم",
    "گرفتهاند": "گرفته‌اند",
    "جامعهشان": "جامعه‌شان",
    "بچهی": "بچه‌ی",
    "شیوهی": "شیوه‌ی",
    "بهکار": "به کار",
    "بهتراست": "بهتر است",
    "سروکلهشون": "سر و کلهشون",
    "رسیدهمسرش": "رسید همسرش",
    "پسراهل": "پسر اهل",
    "پروژههای": "پروژه‌های",
    "عاقلان‌هام": "عاقلانه‌ام",
    "گذاشتهاند": "گذاشته‌اند",
    "کردهام": "کرده‌ام",
    "اندازهگیری": "اندازه گیری",
    "یاوهگویی": "یاوه گویی",
    "سازمانهایی": "سازمان‌هایی",
    "نمودهاند": "نموده‌اند",
    "تنهاییآور": "تنهایی آور",
    "قراردهیم": "قرار دهیم",
    "ازشوهرجان": "از شوهر جان",
    "کرهجنوبی": "کره جنوبی",
    "توهینآمیز": "توهین آمیز",
    "فنآوریهایی": "فناوری‌هایی",
    "داشتهاید": "داشته‌اید",
    "شدهایم": "شده‌ایم",
    "نمیفهمم": "نمیفهمم",
    "مثالهایی": "مثال‌هایی",
    "رییسجمهور": "رییس جمهور",
    "مجموعهی": "مجموعه‌ی",
    "درندهاند": "درنده‌اند",
    "امابهش": "اما بهش",
    "بازخواهند": "باز خواهند",
    "برنامههایی": "برنامه‌هایی",
    "یهجا": "یه جا",
    "زگیلهایی": "زگیل‌هایی",
    "وسیلهی": "وسیله‌ی",
    "بهمنیار": "بهمن یار",
    "دادهام": "داده‌ام",
    "بههنگام": "به هنگام",
    "بهدروغ": "به دروغ",
    "دورافتادهترین": "دور افتاده‌ترین",
    "نامهایی": "نامه‌ایی",
    "سهقسمتی": "سه قسمتی",
    "توجهازچیدن": "توجه از چیدن",
    "پیامرسان‌ها": "پیام رسان‌ها",
    "بهمنزاد": "بهمن زاد",
    "نشانههایی": "نشانه‌هایی",
    "راهحل‌های": "راه حل‌های",
    "راهحلهایی": "راه حل‌هایی",
    "راهحلهای": "راه حل‌های",
    "نظرخواهی‌ها": "نظر خواهی‌ها",
    "نظرخواهیها": "نظر خواهی‌ها",
    "کندهی": "کنده‌ی",
    "حرامزاده‌های": "حرام زاده‌های",
    "شبیهسازیهایی": "شبیه سازی‌هایی",
    "مهارتهایی": "مهارت‌هایی",
    "روبهرویشان": "رو به رویشان",
    "برجستهترین": "برجسته‌ترین",
    "نمیفهمیدم": "نمیفهمیدم",
    "دستگاههایی": "دستگاه‌هایی",
    "برادرشوهر": "برادر شوهر",
    "گرسن‌هام": "گرسته‌ام",
    "گرسنههام": "گرسته‌ام",
    "قهوهخوری": "قهوه خوری",
    "دادهاید": "داده‌اید",
    "بهآرامی": "به آرمانی",
    "دانستنیهاست": "دانستنی‌هاست",
    "بهراحتی": "به راحتی",
    "ایدهپردازی": "ایده‌پردازی",
    "ریشسفیدهای": "ریش سفید‌های",
    "خفهمون": "خفه مون",
    "بهجای": "به جای",
    "ریزخشونت‌ها": "ریز خشونت‌ها",
    "ریزخشونتها": "ریز خشونت‌ها",
    "حساسیتهایی": "حساسیت‌هایی",
    "پشتصحنهی": "پشت صحنه‌ی",
    "کلهی": "کله‌ی",
    "تاشوهرم": "تا شوهرم",
    "آیندهاش": "آینده‌اش",
    "پروانههایی": "پروانه‌هایی",
    "خوبیهایی": "خوبی‌هایی",
    "نرمافزارها": "نرم‌افزارها",
    "رساندهاند": "رسانده‌اند",
    "سرمایهگذارنی": "سرمایه گذارانی",
    "تکهچسبانی": "تکه چسبانی",
    "بیتوجهی": "بی توجهی",
    "جاهطلبی": "جاه طلبی",
    "پرغلغلهتان": "پر غلغله‌تان",
    "خمینیشهر": "خمینی شهر",
    "رشتهتوییت": "رشته توییت",
    "موهبتهایی": "موهبت‌هایی",
    "برنامهی": "برنامه‌ی",
    "مادرشوهردارم": "مادر شوهر داردم",
    "سیاهپوستان": "سیاه پوستان",
    "شرکتهایی": "شرکت‌هایی",
    "نیاوردهاند": "نیاورده‌اند",
    "آنهم": "آن هم",
    "شوهرداریم": "شوهر داریم",
    "یکچهارم": "یک چهارم",
    "پروندههاست": "پرونده هاست",
    "برنامهت": "برنامه‌ات",
    "چروکیدهمان": "چروکیده‌مان",
    "زمینهسازی": "زمینه سازی",
    "زدهاند": "زده‌اند",
    "اظهارنظرپرداختن": "اظهار نظر پرداختن",
    "صلحطلبانهترین": "صلح طلبانه‌ترین",
    "بهغلط": "به غلط",
    "ایدهآلم": "ایده آلم",
    "سیاهکاران": "سیاه کاران",
    "امیرابراهیم": "امیر ابراهیم",
    "توسعهدهندگان": "توسعه دهندگان",
    "لحظهی": "لحظه‌ی",
    "امینطاها": "امین طاها",
    "بینالنهرین": "بین النهرین",
    "نیمهوقت": "نیمه وقت",
    "پیادهروی": "پیاده روی",
    "آلودهاند": "آلوده‌اند",
    "گریهکرد": "گره کرد",
    "نعمتهایی": "نعمت‌هایی",
    "مادرشوهرشماهم": "مادر شوهر شما هم",
    "آشپزخونهاس": "آشپزخونه‌اس",
    "مسابقهها": "مسابقه‌ها",
    "مسابقهای": "مسابقه‌های",
    "برنامهریزی": "برنامه‌ریزی",
    "بازخواهید": "باز خواهید",
    "جوییما": "جویی ما",
    "آماده ایم": "آماده‌ایم",
    "مدلسازی": "مدل‌سازی",
    "درصورتیکه": "در صورتیکه",
    "آمریکاییات": "آمریکایی‌ات",
    "مادریاش": "مادری‌اش",
    "غافلگیرکننده": "غافلگیر کننده",
    "پیکرتراشی": "پیکر تراشی",
    "اذیتوآزار": "اذیت و آزار",
    "امتیازاورترین": "امتیاز آور",
    "جیکجیک": "جیک جیک",
    "تاشب": "تا شب",
    "کپیرایت": "کپی رایت",
    "آنتیبادی": "آنتی بادی",
    "عجیبتر": "عجیب‌تر",
    "استانداردسازی": "استاندارد سازی",
    "هشتادوهشت": "هشتاد و هشت",
    "متنوعتر": "متنوع‌تر",
    "منظورانجام": "منظور انجام",
    "نگرانکننده‌ترین": "نگران کننده‌ترین",
    "شگفتانگیز": "شگفت انگیز",
    "رنگینپوست": "رنگین پوست",
    "فارغ التحصیلان": "فارغ‌التحصیلان",
    "ترسناکتر": "ترسناک‌تر",
    "لا رامبلا": "لارامبلا",
    "پرجمعیتترین": "پرجمعیت‌ترین",
    "درمیآیند": "درمی‌آیند",
    "باشمالکی": "باشم الکی",
    "وسیعتر": "وسیع‌تر",
    "فاحشهخانه": "فاحشه خانه",
    "بااحتیاط": "با احتیاط",
    "قانعکننده": "قانع‌کننده",
    "انعطافپذیری": "انعطاف‌پذیری",
    "بیتالمقدس": "بیت‌المقدس",
    "اوپناستریتمپ": "اوپن استریت مپ",
    "روزابارونی": "روزا بارونی",
    "محافظهکارانه": "محافظه کارانه",
    "فوتبالدستی": "فوتبال دستی",
    "توسعهدهنده": "توسعه دهنده",
    "قانونگزاران": "قانون گزاران",
    "العسریسرا": "العسر یسرا",
    "خارقالعاده": "خارق‌العاده",
    "بیماریمزمن": "بیماری مزمن",
    "بادوستانتان": "با دوستانتان",
    "برابربیشتر": "برابر بیشتر",
    "ارائهدهنده": "ارائه دهنده",
    "طوفانزدگان": "طوفان زندگان",
    "امینمحمد": "امین محمد",
    "محیطزیست": "محیط زیست",
    "شقیترینشان": "شقی‌ترینشان",
    "بودواقعا": "بود واقعا",
    "نیویورکتایمز": "نیویورک تایمز",
    "ریودوژانیرو": "ریو دو ژانیرو",
    "مشترکالمنافع": "مشترک‌المنافع",
    "اسلایدسازم": "اسلاید سازم",
    "نمیآوریدش": "نمی‌آوریدش",
    "بینالملل": "بین‌الملل",
    "مصرفکنندگان": "مصرف کنندگان",
    "امینالدین": "امین الدین",
    "امریکااینقدر": "امریکا اینقدر",
    "بعضیاوقات": "بعضی اوقات",
    "خاطربچه": "خاطر بچه",
    "ایناکیلویی": "اینا کیلویی",
    "ویکیپدیا": "ویکی‌پدیا",
    "مافکرمیکنیم": "ما فکر میکنیم",
    "انگلیسیزبان": "انگلیسی زبان",
    "کلهشون": "کله‌شون",
    "آدمبزرگی": "آرم بزرگی",
    "مر آت مر آه": "مر‌آت مر‌آت",
    "آسیبزد": "آسیب زد",
    "آیآرسی": "آی آرسی",
    "آسیااقیانوسیه": "آسیا اقیانوسیه",
    "آیای": "آیا",
    "میانجنسی": "میان جنسی",
    "میاننسلی": "میان نسلی",
    "میان‌افزار‌ها": "میان افزارها",
    "آییننامه": "آیین‌نامه",
    "ارائهشده": "ارائه‌شده",
    "اشپزخونه": "آشپزخونه",
    "اماعلتشونمیپرسه": "اما علتشو نمیپرسه",
    "امیدوارکننده": "امیدوار کننده",
    "ایالاتمتحده": "ایالات متحده",
    "بااینکه": "با اینکه",
    "بلندپروازانه": "بلند پروازانه",
    "بهترازاینه": "بهتر از اینه",
    "بهدست‌آمده": "به دست‌آمده",
    "بهوسیله": "به وسیله",
    "بیادبانه": "بی ادبانه",
    "بیاندازه": "بی اندازه",
    "بیصبرانه": "بی صبرانه",
    "بیفایده": "بی فایده",
    "بیمهره": "بی مهره",
    "بینظیره": "بی نظیره",
    "تاریخزده": "تاریخ زده",
    "تهرانزده": "تهران زده",
    "تولیدشده": "تولید شده",
    "تولیدکننده": "تولید کننده",
    "تکمیلشده": "تکمیل شده",
    "جاافتاده": "جا افتاده",
    "جمع‌آوریکننده": "جمع‌ آوری کننده",
    "جورآدمیه": "جور آدمیه",
    "حقالزحمه": "حق الزحمه",
    "دخترونهتره": "دخترونه تره",
    "دوپنجره": "دو پنجره",
    "ذاتالریه": "ذات‌الریه",
    "راسالخیمه": "راس‌الخیمه",
    "رنگماده": "رنگ ماده",
    "سوئاستفاده": "سو استفاده",
    "سواستفاده": "سو استفاده",
    "شبهجزیره": "شبه جزیره",
    "صادرکننده": "صادر کننده",
    "ضررداره": "ضرر داره",
    "عابرپیاده": "عابر پیاده",
    "فوقالعاده": "فوق‌العاده",
    "قابلتوجه": "قابل توجه",
    "قانع‌کننده": "قانع‌ کننده",
    "مادربیچاره": "مادر بیچاره",
    "مشخصشده": "مشخص شده",
    "مصرفکننده": "مصرف کننده",
    "مصیبتزده": "مصیب تزده",
    "ناامیدکننده": "ناامید کننده",
    "نیمفاصله": "نیم‌فاصله",
    "هماهنگکننده": "هماهنگ کننده",
    "همهجانبه": "همه جانبه",
    "واردکننده": "وارد کننده",
    "وخوابگاه": "و خوابگاه",
    "ودستگاه": "و دستگاه",
    "وزردچوبه": "و زردچوبه",
    "وپروانه": "و پروانه",
    "پدرخوانده": "پدر خوانده",
    "چاپشده": "چاپ شده",
    "کردته": "کرد ته",
    "کردندکه": "کردند که",
    "یکطرفه": "یک طرفه",
    "پایینتره": "پایین‌تره",
    "اشتراکگذاری": "اشتراک گذاری",
    "انحصارگراناند": "انحصار گران‌اند",
    "خوشحالییییی": "خوشحالی",
    "همتیمی‌هایشان": "هم تیمی‌هایشان",
    "پایدار‌ام‌باید": "پایدار‌ام ‌باید",
    "پرجنبوجوش‌تر": "پر جنب و جوش‌تر",
    "آبمروارید": "آب مروارید",
    "آتشسوزی": "آتش سوزی",
    "آتشنشانی": "آتش‌نشانی",
    "آتشنشان": "آتش‌نشان",
    "آرامشبخش": "آرامش بخش",
    "آشناداشتن": "آشنا داشتن",
    "آقاچیزی": "آقا چیزی",
    "آموخت‌هام": "آموخته‌ام",
    "آموزششان": "آموزش‌شان",
    "ازآنجا": "از آنجا",
    "ازالان": "از الان",
    "ازاینجا": "از اینجا",
    "ازجیبش": "از جیبش",
    "ازدستش": "از دستش",
    "ازدیوار": "از دیوار",
    "ازشغلشون": "از شغلشون",
    "ازوقتی": "از وقتی",
    "ازکسانی": "از کسانی",
    "اسباببازی": "اسباب بازی",
    "اسبسوار": "اسب سوار",
    "اصیلزاده": "اصیل زاده",
    "افتادهاید": "افتاده‌اید",
    "ال‌هام": "الهام",
    "امااصلا": "اما اصلا",
    "امااصلابه": "اما اصلا به",
    "امااین": "اما این",
    "امابعد": "اما بعد",
    "امابعدیکی": "اما بعد یکی",
    "اماجاذبه": "اما جاذبه",
    "امرارمعاش": "امرار معاش",
    "امکانپذیر": "امکان پذیر",
    "انت‌های": "انتهای",
    "انت‌هایی": "انتهایی",
    "ایزدبانوی": "ایزد بانوی",
    "بااینحال": "با اینحال",
    "باحتمال": "به احتمال",
    "باحجاب": "با حجاب",
    "باخنده": "با خنده",
    "بادوستاش": "با دوستاش",
    "بارمان": "بار مان",
    "باز‌تر": "باز ‌تر",
    "باطعنه": "با طعنه",
    "بافریاد": "با فریاد",
    "بارگزاری": "بارگذاری",
    "بالامنم": "بالا منم",
    "بگیرمامان": "بگیر مامان",
    "بیاحترامی": "بی احترامی",
    "بیادبی": "بی ادبی",
    "بیاعتنا": "بی اعتنا",
    "بیدارباش": "بیدار باش",
    "بیشازحد": "بیش از حد",
    "بیمسئولیت": "بی مسئولیت",
    "تاسفبار": "تاسف بار",
    "تامشکلمون": "تا مشکلمون",
    "تانقشه": "تا نقشه",
    "تصمیمگیری": "تصمیم گیری",
    "تقسیمبندی": "تقسیم بندی",
    "تقصیرارو": "تقصیرا رو",
    "جدیدابرای": "جدیدا برای",
    "جعبهابزار": "جعبه ابزار",
    "جلوتونو": "جلو تو نو",
    "حاضردر": "حاضر در",
    "حاضرنیست": "حاضر نیست",
    "دستنخورده": "دست نخورده",
    "دوامتیاز": "دو امتیاز",
    "دوروزتمام": "دو روز تمام",
    "شخصیسازی": "شخصی‌سازی",
    "شدواجناس": "شد و اجناس",
    "شوهردارم": "شوهر دارم",
    "شوهرشماهم": "شوهر شما هم",
    "شوهرمحترم": "شوهر محترم",
    "شکلگیری": "شکل گیری",
    "صخرهنوردی": "صخره‌نوردی",
    "صدوبیست": "صد و بیست",
    "عقبنشینی": "عقب نشینی",
    "عکسالعمل": "عکس‌العمل",
    "غرغرمیکنم": "غرغر میکنم",
    "هزاربار": "هزار بار",
    "هزارتومان": "هزار تومان",
    "هزارجور": "هزار جور",
    "هزاروسیصد": "هزار و سیصد",
    "هممیهنان": "هم میهنان",
    "هممیهنانش": "هم میهنانش",
    "همنسلانش": "هم نسلانش",
    "همهگیری": "همه گیری",
    "هییییچ": "هیچ",
    "وقتاخیلی": "وقتا خیلی",
    "وقتابه": "وقتا به",
    "وقتگذرانی": "وقت گذرانی",
    "ومحکوم": "و محکوم",
    "ومحیط‌ها": "و محیط‌ها",
    "وکشورتان": "و کشورتان",
    "ویکیمدیا": "ویکی‌مدیا",
    "یهوگفت": "یهو گفت",
    "اینجااز": "اینجا از",
}
fixator_dictionary = {
    "ب‌های": "بهای",
    "به‌ترین": "بهترین",
    "آس‌تر": "‌آستر",
    "ارکس‌تر": "ارکستر",
    "ان‌تر": "انتر",
    "بس‌تر": "بستر",
    "به‌تر": "بهتر",
    "به‌ترتر": "بهترتر",
    "توئی‌تر": "تویتتر",
    "تویی‌تر": "توییتر",
    "تی‌تر": "تیتر",
    "دخ‌تر": "دختر",
    "دف‌تر": "دفتر",
    "دلس‌تر": "دلستر",
    "دک‌تر": "دکتر",
    "ش‌تر": "شتر",
    "لی‌تر": "لیتر",
    "م‌تر": "متر",
    "هیپس‌تر": "هیپستر",
    "پی‌تر": "پیتر",
    "چ‌تر": "چتر",
    "کم‌تر": "کمتر",
    "گنگس‌تر": "گنگستر",
    "انگش‌تر": "انگشتر",
    "سن‌تر": "سنتر",
    "تویت‌تر": "توییتر",
    "مادهش‌تر": "ماده شتر",
    "وی‌ترین": "ویترین",
    "کرونوم‌تر": "کرنومتر",
    "که‌تر": "کهتر",
    "فیل‌تر": "فیلتر",
    "ال‌هام": "الهام",
    "آل‌مان": "آلمان",
    "انت‌های": "انتهای",
    "انت‌هایی": "انتهایی",
    "آموخت‌هام": "آموخته‌ام",
}

In [ ]:
def cleaning(text):
    if not isinstance(text, str):
        return None

    return normalizer({"sentence": text}, return_dict=False)

In [ ]:
data_dir = "/content/cv-corpus-6.1-2020-12-11/fa"

data = pd.read_csv(f"{data_dir}/validated.tsv", sep='\t')

data = data[:9000]

data = data.drop(['accent', 'age', 'client_id', 'down_votes', 'gender', 'locale', 'segment', 'up_votes'], axis=1)

data["path"] = data_dir + "/clips/" + data["path"]

print(f"Step 0: {len(data)}")

Step 0: 9000


In [ ]:
data["status"] = data["path"].apply(lambda path: True if os.path.exists(path) else None)
data = data.dropna(subset=["path"])
data = data.drop("status", 1)

print(f"Step 1: {len(data)}")

Step 1: 9000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
data["sentence"] = data["sentence"].apply(lambda t: cleaning(t))
data = data.dropna(subset=["sentence"])

print(f"Step 2: {len(data)}")

Step 2: 9000


In [ ]:
data = data.reset_index(drop=True)
display(data.head())

,path,sentence
0,/content/cv-corpus-6.1-2020-12-11/fa/clips/com...,از مهمونداری کنار بکشم
1,/content/cv-corpus-6.1-2020-12-11/fa/clips/com...,برو از مهرداد بپرس
2,/content/cv-corpus-6.1-2020-12-11/fa/clips/com...,خب تو چیکار می کنی
3,/content/cv-corpus-6.1-2020-12-11/fa/clips/com...,مسقط پایتخت عمان در عربی به معنای محل سقوط است
4,/content/cv-corpus-6.1-2020-12-11/fa/clips/com...,آه نه اصلا


In [ ]:
train = data[["path", "sentence"]][:7500]
test = data[["path", "sentence"]][8000:9000]

train.to_csv("/content/train.csv", sep='\t', encoding="utf-8", index=False)
test.to_csv("/content/test.csv", sep='\t', encoding="utf-8", index=False)

In [ ]:
common_voice_train = load_dataset('csv', data_files={'train': 'train.csv'}, delimiter='\t')['train']
common_voice_test = load_dataset('csv', data_files={'test': 'test.csv'}, delimiter='\t')['test']

Using custom data configuration default-ffbb255037e28856


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-ffbb255037e28856/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-36cd40b39810e259


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-36cd40b39810e259/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(common_voice_train)
print(common_voice_test)

Dataset({
    features: ['path', 'sentence'],
    num_rows: 7500
})
Dataset({
    features: ['path', 'sentence'],
    num_rows: 1000
})


In [ ]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), 'Can\'t pick more elements than there are in the dataset.'

    picks = []

    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)

        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(df.head())

In [ ]:
show_random_elements(common_voice_train.remove_columns(['path']), num_examples=10)

,sentence
0,مایک هم برا شام میاد
1,وقتی از حقوقت راضی نیستی
2,دانشمندی از انجا میگذشت از طاووس پرسید چرا پر‌...
3,منتظرم که آبنبات بکیرم
4,من گل کلم را به بروکلی ترجیح می دهم


In [ ]:
print(common_voice_train[0]['sentence'])
print(common_voice_test[0]['sentence'])

از مهمونداری کنار بکشم 
نگه داری میکنه 


In [ ]:
def extract_all_chars(batch):
    all_text = ' '.join(batch['sentence'])
    vocab = list(set(all_text))
    return {'vocab': [vocab], 'all_text': [all_text]}

In [ ]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(sorted(set(vocab_train['vocab'][0]) | set(vocab_test['vocab'][0])))
vocab_list = [vocab for vocab in vocab_list if vocab not in [' ', '\u0307']]
print(len(vocab_list))
print(vocab_list)

35
['آ', 'ئ', 'ا', 'ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق', 'ل', 'م', 'ن', 'ه', 'و', 'پ', 'چ', 'ژ', 'ک', 'گ', 'ی', '\u200c']


In [ ]:
special_vocab = ['<pad>', '<s>', '</s>', '<unk>', '|']
vocab_dict = {v: k for k, v in enumerate(special_vocab + vocab_list)}
print(len(vocab_dict))
print(vocab_dict)

40
{'<pad>': 0, '<s>': 1, '</s>': 2, '<unk>': 3, '|': 4, 'آ': 5, 'ئ': 6, 'ا': 7, 'ب': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ل': 28, 'م': 29, 'ن': 30, 'ه': 31, 'و': 32, 'پ': 33, 'چ': 34, 'ژ': 35, 'ک': 36, 'گ': 37, 'ی': 38, '\u200c': 39}


In [ ]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
tokenizer = Wav2Vec2CTCTokenizer('vocab.json', bos_token='<s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>', word_delimiter_token='|',
                                 do_lower_case=False)

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [ ]:
text = 'از مهمونداری کنار بکشم'
print(' '.join(tokenizer.tokenize(text)))
print(tokenizer.decode(tokenizer.encode(text)))

ا ز | م ه م و ن د ا ر ی | ک ن ا ر | ب ک ش م
از مهمونداری کنار بکشم


In [ ]:
if len(processor.tokenizer.get_vocab()) == len(processor.tokenizer):
    print(len(processor.tokenizer))

40


In [ ]:
target_sampling_rate = 16_000

min_duration_in_seconds = 5.0
max_duration_in_seconds = 10.0

In [ ]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch['path'])
    speech_array = speech_array.squeeze().numpy()
    speech_array = librosa.resample(np.asarray(speech_array), sampling_rate, target_sampling_rate)

    batch['speech'] = speech_array
    batch['sampling_rate'] = target_sampling_rate
    batch['duration_in_seconds'] = len(batch['speech']) / target_sampling_rate
    batch['target_text'] = batch['sentence']
    return batch

def filter_by_max_duration(batch):
    return min_duration_in_seconds <= batch['duration_in_seconds'] <= max_duration_in_seconds

# check that all files have the correct sampling rate
def prepare_dataset(batch):
    assert (
        len(set(batch['sampling_rate'])) == 1), f'Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}.'

    batch['input_values'] = processor(batch['speech'], sampling_rate=batch['sampling_rate'][0]).input_values

    with processor.as_target_processor():
        batch['labels'] = processor(batch['target_text']).input_ids

    return batch

In [ ]:
common_voice_train = common_voice_train.map(speech_file_to_array_fn, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(speech_file_to_array_fn, remove_columns=common_voice_test.column_names)

  0%|          | 0/7500 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=48000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=44100, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Pass orig_sr=32000, target_sr=16000 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  after removing the cwd from sy

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [ ]:
print(common_voice_train[0]['sampling_rate'])
print(common_voice_test[0]['sampling_rate'])

16000
16000


In [ ]:
print(f'Split sizes [BEFORE]: {len(common_voice_train)} train and {len(common_voice_test)} validation.')

Split sizes [BEFORE]: 7500 train and 1000 validation.


In [ ]:
_common_voice_train = common_voice_train.filter(filter_by_max_duration)
_common_voice_test = common_voice_test.filter(filter_by_max_duration)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
print(f'Split sizes [AFTER]: {len(_common_voice_train)} train and {len(_common_voice_test)} validation.')

Split sizes [AFTER]: 3342 train and 415 validation.


In [ ]:
_common_voice_train = _common_voice_train.map(prepare_dataset, remove_columns=_common_voice_train.column_names, batch_size=8, batched=True)
_common_voice_test = _common_voice_test.map(prepare_dataset, remove_columns=_common_voice_test.column_names, batch_size=8, batched=True)

  0%|          | 0/418 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)


  0%|          | 0/52 [00:00<?, ?ba/s]

In [ ]:
_common_voice_train.set_format(type='torch', columns=['input_values', 'labels'])
_common_voice_test.set_format(type='torch', columns=['input_values', 'labels'])

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{'input_values': feature['input_values']} for feature in features]
        label_features = [{'input_ids': feature['labels']} for feature in features]

        batch = self.processor.pad(input_features, padding=self.padding, max_length=self.max_length, pad_to_multiple_of=self.pad_to_multiple_of, return_tensors='pt')

        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(label_features, padding=self.padding, max_length=self.max_length_labels, pad_to_multiple_of=self.pad_to_multiple_of_labels, return_tensors='pt')

        labels = labels_batch['input_ids'].masked_fill(labels_batch.attention_mask.ne(1), -100)
        batch['labels'] = labels
        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
def compute_metrics(pred):
    
    pred_logits = pred.predictions
    print('pred_logits', pred_logits)
    pred_ids = np.argmax(pred_logits, axis=-1)
    print('pred_ids',pred_ids)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    print('pred_str',pred_str)

    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    print('label_str',label_str)

    if isinstance(label_str, list):
        if isinstance(pred_str, list) and len(pred_str) == len(label_str):
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str[index]}')
        else:
            for index in random.sample(range(len(label_str)), 3):
                print(f'reference: {label_str[index]}')
                print(f'predicted: {pred_str}')

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {'wer': wer}

In [ ]:
wer_metric = load_metric('wer')

In [ ]:
print(len(processor.tokenizer.get_vocab()))

40


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-large-xlsr-53', attention_dropout=0.1, hidden_dropout=0.1, feat_proj_dropout=0.0,
                                       mask_time_prob=0.05, layerdrop=0.1, gradient_checkpointing=True, ctc_loss_reduction='mean', 
                                       ctc_zero_infinity=True, bos_token_id=processor.tokenizer.bos_token_id, eos_token_id=processor.tokenizer.eos_token_id,
                                       pad_token_id=processor.tokenizer.pad_token_id, vocab_size=len(processor.tokenizer.get_vocab()))

model.freeze_feature_extractor()

model.to(torch.device('cuda'))

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_hid.bias', 'project_q.bias', 'quantizer.weight_proj.weight', 'project_q.weight', 'quantizer.codevectors', 'quantizer.weight_proj.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [ ]:
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/spu', group_by_length=True, per_device_train_batch_size=10, per_device_eval_batch_size=10,
                                  save_strategy='no', eval_steps=10, gradient_accumulation_steps=2, evaluation_strategy='steps', num_train_epochs=1,
                                  learning_rate=1e-4, no_cuda=False, fp16=True, logging_steps=10, warmup_steps=1, save_total_limit=1,)
                                  #load_best_model_at_end=True), save_steps=10,

trainer = Trainer(model=model, data_collator=data_collator, args=training_args, compute_metrics=compute_metrics, train_dataset=_common_voice_train,
                  eval_dataset=_common_voice_test, tokenizer=processor.feature_extractor)

Using amp half precision backend


In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3342
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 2
  Total optimization steps = 167


Step,Training Loss,Validation Loss,Wer
10,22.743800,20.542463,1.007119
20,20.925400,18.347382,1.000000
30,16.052200,10.397763,1.000000
40,11.379700,5.189136,1.000000
50,5.761400,4.004157,1.000000
60,4.206600,3.574928,1.000000
70,3.647800,3.376488,1.000000
80,3.450700,3.251455,1.000000
90,3.261700,3.160048,1.000000
100,3.141700,3.101135,1.000000


***** Running Evaluation *****
  Num examples = 415
  Batch size = 10


reference: نه خوابیده بچم انقدر که خسته و کلافست
predicted: </s> م م م
reference: با استفاده از ذهن تحلیلی لورا و بینش انسان محور من
predicted: </s> م ب ب
reference: واقعا پشمام از این کلیپ واقعا پشمام از این نسل جدید
predicted: </s> م م  ب ب ب ب


***** Running Evaluation *****
  Num examples = 415
  Batch size = 10


...

***** Running Evaluation *****
  Num examples = 415
  Batch size = 10


reference: تقریبا هر منطقه‌ای از جهان نوشیدنی تخمیر شده خود را داشت
predicted: 
reference: چطور دلتون میاد این هر چی هم که باشه باز هم یه موشه یه موش بدبختو مفلوکه
predicted: 
reference: و من لازم دارم برای خواندن آن عینک مطالعه خود را بردارم
predicted: 


***** Running Evaluation *****
  Num examples = 415
  Batch size = 10


reference: ماداگاسکار چهارمین جزیره بزرگ جهان میباشد
predicted: 
reference: من آقای فلاحی هستم
predicted: 
reference: مزدک میرزایی گزارشگر بازی حساس پرسپولیس کاشیما شد
predicted: 




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
metrics = train_result.metrics
max_train_samples = len(_common_voice_train)
metrics['train_samples'] = min(max_train_samples, len(_common_voice_train))

trainer.save_model()

trainer.log_metrics('train', metrics)
trainer.save_metrics('train', metrics)
trainer.save_state()

Saving model checkpoint to /content/drive/MyDrive/spu
Configuration saved in /content/drive/MyDrive/spu/config.json
Model weights saved in /content/drive/MyDrive/spu/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/spu/preprocessor_config.json


***** train metrics *****
  epoch                    =         1.0
  total_flos               = 625603066GF
  train_loss               =      6.9035
  train_runtime            =  0:12:46.49
  train_samples            =        3342
  train_samples_per_second =        4.36
  train_steps_per_second   =       0.218


In [ ]:
model = Wav2Vec2ForCTC.from_pretrained('facebook/wav2vec2-large-xlsr-53', attention_dropout=0.1, hidden_dropout=0.1, feat_proj_dropout=0.0,
                                       mask_time_prob=0.05, layerdrop=0.1, gradient_checkpointing=True, ctc_loss_reduction='mean', 
                                       ctc_zero_infinity=True, bos_token_id=processor.tokenizer.bos_token_id, eos_token_id=processor.tokenizer.eos_token_id,
                                       pad_token_id=processor.tokenizer.pad_token_id, vocab_size=len(processor.tokenizer.get_vocab()))

model.freeze_feature_extractor()

model.to(torch.device('cuda'))

loading configuration file https://huggingface.co/facebook/wav2vec2-large-xlsr-53/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/8508c73cd595eb416a1d517b90762416c0bc6cfbef529578079aeae4d8c14336.7581ed2ee0c677f1e933180df51bd1a668c4a2b6d5fd1297d32069373dac097c
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "d

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (1): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (2): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation): GELUActivation()
        )
        (3): Wav2Vec2LayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,))
          (layer_norm): LayerNorm((512,), eps=1e-05, elemen

In [ ]:
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/spu2', group_by_length=True, per_device_train_batch_size=10, per_device_eval_batch_size=10,
                                  save_strategy='no', eval_steps=10, gradient_accumulation_steps=2, evaluation_strategy='steps', num_train_epochs=1,
                                  learning_rate=1e-3, no_cuda=False, fp16=True, logging_steps=10, warmup_steps=1, save_total_limit=1,)
                                  #load_best_model_at_end=True), save_steps=10,

trainer = Trainer(model=model, data_collator=data_collator, args=training_args, compute_metrics=compute_metrics, train_dataset=_common_voice_train,
                  eval_dataset=_common_voice_test, tokenizer=processor.feature_extractor)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using amp half precision backend


In [ ]:
train_result = trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3342
  Num Epochs = 1
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 2
  Total optimization steps = 167


RuntimeError: ignored